# Republic frase modellen

Doel:

- informatie extractie uit de resoluties, gebruik makend van de repetitieve aspecten van de teksten, formulaische uitdrukkingen, signaalwoorden en onderwerpstermen.
- het opbouwen van collectie-specifieke frase-modellen (lijstjes met termen en frasen, varianten daarvan en metadata categorieën en labels) zodat we informatie kunnen extraheren en om kunnen vormen tot inhoudelijke metadata voor ontsluiting van de individuele resoluties. 

Aanpak: harvesting phrases o.b.v. frequentielijsten

Verschillen tussen jaren:

- tussen twee opvolgende jaren
- tussen twee jaren met een decennium ertussen
- tussen twee jaren met X jaren ertussen


- tussen automatische extractie en respecten (verhouding tussen index en automatische extractie). Dit vergt ook interactie met Team Text en NER activiteiten

## Overlap analyseren

**doel:** bepalen welke termen structureel vaak voorkomen en hoe ze kunnen worden gebruikt om de inhoud van resoluties te analyseren

Lijstjes:

- hoedanigheden, rollen (ambassadeur, graaf, suppliant)
- signaalwoorden:
    - <ambassadeur> in, uit, van, bij <land/hof>
    - uit, naar, tot, te <plaats>
    - gecommiteerd/commissie


In [14]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_dir = os.path.split(os.getcwd())[0]
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

In [17]:
from typing import Union
from republic.model.inventory_mapping import get_inventory_by_num
from republic.config.republic_config import base_config, set_config_inventory_num
import republic.elastic.republic_elasticsearch as rep_es

def make_match_inventory_query(inv_num: int) -> dict:
    return {
        "query": {
            "bool": {
                "must": [
                    {"match": {"metadata.inventory_num": inv_num}},
                ]
            }
        },
        "size": 0
    }

def get_keyword_context(text: str, keyword: str, 
                        context_size: int = 3, 
                        prefix_size: Union[None, int] = None, 
                        suffix_size: Union[None, int] = None):
    if not prefix_size:
        prefix_size = context_size
    if not suffix_size:
        suffix_size = context_size
    text = re.sub(r'\s+',' ', text)
    prefix_pattern = r'(\w*\W*){' + f'{prefix_size}' + '}$'
    suffix_pattern = r'^(\w*\W*){' + f'{suffix_size}' + '}'
    contexts = []
    for match in re.finditer(r'\b' + keyword + r'\b', text):
        prefix_window = text[:match.start()]
        suffix_window = text[match.end():]
        prefix_terms = prefix_window.strip().split(' ')
        suffix_terms = suffix_window.strip().split(' ')
        prefix = ' '.join(prefix_terms[-prefix_size:])# re.search(prefix_pattern, prefix_window)
        suffix = ' '.join(suffix_terms[:suffix_size])# re.search(suffix_pattern, suffix_window)
        context = {
            'keyword': keyword,
            'prefix': prefix,
            'suffix': suffix,
            'keyword_offset': match.start(),
            'prefix_offset': match.start() - len(prefix),
            'suffix_offset': match.end()
        }
        contexts.append(context)
    return contexts
        



es_republic = rep_es.initialize_es(host_type='external', timeout=60)



In [18]:
import re
from collections import Counter

inv_num = 3761
query = make_match_inventory_query(inv_num)
query["size"] = 10000
paragraph_index = "republic_hocr_paragraphs"

response = es_republic.search(index=paragraph_index, body=query)
print(f"Total hits: {response['hits']['total']['value']}\n")

docs = [hit['_source'] for hit in response['hits']['hits']]
title_freq = Counter()

for doc in docs:
    terms = re.split(r'\W+', doc['text'])
    title_terms = [term for term in terms if term != '' and term[0].isupper()]
    title_freq.update(title_terms)

    
for title, freq in title_freq.most_common(150):
    print(f'{title: <30}{freq: >6}')

Total hits: 2889

Hoogh                           4898
Mogende                         4591
Heeren                          2061
Waer                            1862
Requefte                        1527
Vergaderinge                    1437
Heere                           1270
Gedeputeerden                   1125
Suppliant                       1049
Miflive                         1047
Is                              1008
Raedt                            915
Ontfangen                        894
Refolutie                        771
State                            685
Op                               643
Supplianten                      567
Regiment                         564
Paerden                          559
A                                499
Staten                           472
Provincie                        435
Copie                            430
Pafport                          419
Majefteyt                        406
Stadt                            406
De                  

In [20]:
prefix_freq = Counter()
suffix_freq = Counter()

term = 'Schip'
term = 'Collegie'
term = 'Gecommitteerden'
term = 'Regiment'
term = 'Compagnie'
term = 'Regenten'
term = 'Pafport'
term = 'Placaat'
term = 'ophef'
term = 'Ambaffadeur' # (van|aan het hof van|tot)
term = 'Raedt'

for doc in docs:
    for context in get_keyword_context(doc['text'], term, context_size=5):
        prefix_freq.update([context['prefix']])
        suffix_freq.update([context['suffix']])
    
        
for prefix, freq in prefix_freq.most_common(10):
    print(f'{prefix: <30}{freq: >6}')

print('\n')

for suffix, freq in suffix_freq.most_common(100):
    print(f'{suffix: <30}{freq: >6}')



eenige Heeren Gecommitteerden uyt den    83
Ontfangen een Miflive van den     30
gefonden {al werden aen den       24
gefonden fal werden aen den       20
cenige Heeren Gecommitteerden uyt den    16
Ontfangen cen Miflive van den      8
Ontfangen een Miffive van den      7
gefonden (al werden aen den        6
regiftreren ter Secretarye van den     6
nevens eenige Heeren Gecommitteerden uytden     5


van State, by haer E.             33
van State gefchreven alhier in    18
van State by haer E.              18
van State, gefchreven alhier in    16
van State, om aen haer            15
van State ‚ by haer                9
van State , by haer                8
van State om aen haer              8
van State, geëxamineert de Miflive     7
van State ende des Generaliteyts     5
van Staten by haer E.              4
van State, om te {trecken          4
van State, geëxamineert de Miffive     4
van gefchreven alhier in den       4
van State, ende het felve          4
van Staten gefchreven alhier in